In [23]:
import time 
import logging
import theano
import scipy as sp
from theano import tensor as tt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm

plt.style.use('seaborn-darkgrid')

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [19]:
data = pd.read_csv('../data/movie.data', sep='\t', names=["userid", "itemid", "rating", "timestamp"])

movie_columns  = ['movie id', 'movie title', 'release date', 'video release date', 'IMDb URL',
                  'unknown','Action','Adventure', 'Animation',"Children's", 'Comedy', 'Crime',
                  'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                  'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('../data/u.item', sep='|', names=movie_columns , encoding='latin-1',index_col="movie id",parse_dates=['release date'])
ratings = data.rating # 영화 ratings

In [20]:
df = data.pivot_table('rating', index='userid', columns='itemid').values

In [2]:
class PMF():
    
    def __init__(self, train, dim , alpha=2, std=0.01, bounds=(1,5)):
        self.dim = dim # latent dimension
        self.alpha = alpha # likeihood variance : sigma
        self.std = np.sqrt(1/alpha) # 
        self.bounds = bounds
        self.data = train.copy()
        n, m = self.data.shape 
        
        nan_mask = np.isnan(self.data) # missing mask
        self.data[nan_mask] = self.data[~nan_mask].mean() # 평균값으로 nan 값 채워넣음
        
        self.alpha_u = 1/ self.data.var(axis=1).mean() # u 분산 평균
        self.alpha_v = 1/ self.data.var(axis=0).mean() # v 분산 평균
        
        logging.info('building the PMF model')
        with pm.Model() as pmf:
            """
            
            """
            U = pm.MvNormal('U', mu = 0, tau = self.alpha_u * np.eye(dim),
                           shape = (n,dim), testval = np.random.randn(n,dim) * std)
            V = pm.MvNormal('V', mu = 0, tau = self.alpha_v * np.eye(dim),
                            shape = (m,dim), testval=np.random.randn(m,dim)*std)
            R = pm.Normal(
                'R', mu= tt.dot(U, V.T)[~nan_mask], tau=self.alpha,
                observed=self.data[~nan_mask])
    
        logging.info('done building the PMF model')
        self.model = pmf
    def __str__(self):
        return self.name


In [3]:
def _find_map(self):
    tstart = time.time()
    with self.model:
        logging.info('finding PMF MAP using L-bfgs-b optimization...')
        self._map = pm.find_MAP(method='L-BFGS-B')
    elapsed = int(time.time()- tstart)
    logging.info('found PMF MAP in %d seconds'% elapsed)
    return self._map

def _map(self):
    try:
        return self._map
    except :
        return self.find_map()
    
PMF.find_map = _find_map
PMF.map = property(_map)

In [4]:
def _draw_samples(self, **kwargs):
    kwargs.setdefault('chains', 1)
    with self.model:
        self.trace =pm.sample(**kwargs)
PMF.draw_samples = _draw_samples

In [5]:
def _predict(self, U, V):
    R = np.dot(U, V.T)
    n,m = R.shape
    sample_R = np.random.normal(R, self.std) # 
    
    low, high = self.bounds
    sample_R[sample_R< low] = low
    sample_R[sample_R> high] = high
    return sample_R
PMF.predict = _predict

In [6]:
def rmse(test_data, predicted):
    I = ~np.isnan(test_data)
    N = I.sum()
    sqerror = abs(test_data - predicted)**2
    mse = sqerror[I].sum()/N
    return np.sqrt(mse)

In [21]:
def split_train_test(data, percent_test=0.1):
    n, m = data.shape
    N = n * m
    
    train = data.copy()
    test = np.ones(data.shape) * np.nan
    
    tosample = np.where(~np.isnan(train)) 
    idx_pairs = list(zip(tosample[0], tosample[1]))
    
    test_size = int(len(idx_pairs) * percent_test)
    train_size = len(idx_pairs)- test_size
    
    indices = np.arange(len(idx_pairs))
    sample = np.random.choice(indices, replace=False, size=test_size)
    
    for idx in sample:
        idx_pair = idx_pairs[idx]
        test[idx_pair] = train[idx_pair]  # transfer to test set
        train[idx_pair] = np.nan          # remove from train set

    # Verify everything worked properly
    assert(train_size == N-np.isnan(train).sum())
    assert(test_size == N-np.isnan(test).sum())

    # Return train set and test set
    return train, test

train, test = split_train_test(df)

In [24]:
ALPHA = 2
DIM = 10
pmf = PMF(train, DIM, ALPHA, std=0.05)

INFO:root:building the PMF model
C:\Users\dhdcj\AppData\Roaming\Python\Python37\site-packages\theano\tensor\subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
INFO:root:done building the PMF model


In [25]:
pmf.find_map()

INFO:root:finding PMF MAP using L-bfgs-b optimization...
C:\Users\dhdcj\AppData\Roaming\Python\Python37\site-packages\pymc3\tuning\starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')
  0%|                                                                                         | 0/5000 [00:00<?, ?it/s]C:\Users\dhdcj\AppData\Roaming\Python\Python37\site-packages\theano\tensor\subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
l

{'U': array([[ 0.82505808, -1.45492477,  0.05392429, ...,  0.14327027,
          0.16780184, -0.29148398],
        [ 0.01519386, -1.22141107, -0.20036465, ...,  0.51351686,
          0.40611433, -0.28093899],
        [ 0.3733684 , -1.29511723,  0.02166351, ...,  0.44597149,
          0.08276389,  0.02716791],
        ...,
        [ 0.20036194, -0.96898937,  0.20793147, ...,  0.51368997,
          0.09223361, -0.42661118],
        [-0.50060905, -1.28093761, -0.30659359, ...,  1.15503589,
          0.00503589, -0.19902283],
        [-0.18513846, -1.30110614, -0.32309749, ...,  0.02112201,
         -0.07602124, -0.89652119]]),
 'V': array([[ 0.03241338, -1.12977888,  0.140434  , ...,  0.9328965 ,
         -0.11263378, -1.04692399],
        [-0.34222624, -0.96291915,  0.11415688, ...,  0.59545962,
         -0.06977058, -0.66270539],
        [-0.12706489, -0.93052194, -0.47305359, ...,  0.12968848,
         -0.03594224,  0.74588257],
        ...,
        [-0.00217322, -0.17472074, -0.079047

In [26]:
def eval_map(pmf_model, train, test):
    U = pmf_model.map['U']
    V = pmf_model.map['V']

    # Make predictions and calculate RMSE on train & test sets.
    predictions = pmf_model.predict(U, V)
    train_rmse = rmse(train, predictions)
    test_rmse = rmse(test, predictions)
    overfit = test_rmse - train_rmse

    # Print report.
    print('PMF MAP training RMSE: %.5f' % train_rmse)
    print('PMF MAP testing RMSE:  %.5f' % test_rmse)
    print('Train/test difference: %.5f' % overfit)

    return test_rmse


# Add eval function to PMF class.
PMF.eval_map = eval_map

In [27]:
pmf_map_rmse = pmf.eval_map(train, test)
pmf_improvement = baselines['mom'] - pmf_map_rmse
print('PMF MAP Improvement:   %.5f' % pmf_improvement)

PMF MAP training RMSE: 1.00948
PMF MAP testing RMSE:  1.13847
Train/test difference: 0.12899


NameError: name 'baselines' is not defined

In [28]:
pmf.draw_samples(draws=500, tune=500)

INFO:pymc3:Auto-assigning NUTS sampler...
INFO:pymc3:Initializing NUTS using jitter+adapt_diag...
C:\Users\dhdcj\AppData\Roaming\Python\Python37\site-packages\theano\tensor\subtensor.py:2197: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rval = inputs[0].__getitem__(inputs[1:])
INFO:pymc3:Sequential sampling (1 chains in 1 job)
INFO:pymc3:NUTS: [V, U]
100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [2:03:18<00:00,  5.12s/it]
INFO:pymc3:Only one chain was sampled, this makes it impossible to run some convergence checks


In [ ]:
def _norms(pmf_model, monitor=('U', 'V'), ord='fro'):
    """Return norms of latent variables at each step in the
    sample trace. These can be used to monitor convergence
    of the sampler.
    """
    monitor = ('U', 'V')
    norms = {var: [] for var in monitor}
    for sample in pmf_model.trace:
        for var in monitor:
            norms[var].append(np.linalg.norm(sample[var], ord))
    return norms


def _traceplot(pmf_model):
    """Plot Frobenius norms of U and V as a function of sample #."""
    trace_norms = pmf_model.norms()
    u_series = pd.Series(trace_norms['U'])
    v_series = pd.Series(trace_norms['V'])
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    u_series.plot(kind='line', ax=ax1, grid=False,
                  title="$\|U\|_{Fro}^2$ at Each Sample")
    v_series.plot(kind='line', ax=ax2, grid=False,
                  title="$\|V\|_{Fro}^2$ at Each Sample")
    ax1.set_xlabel("Sample Number")
    ax2.set_xlabel("Sample Number")


PMF.norms = _norms
PMF.traceplot = _traceplot

In [ ]:
pmf.traceplot()

In [ ]:
def _running_rmse(pmf_model, test_data, train_data, burn_in=0, plot=True):
    """Calculate RMSE for each step of the trace to monitor convergence.
    """
    burn_in = burn_in if len(pmf_model.trace) >= burn_in else 0
    results = {'per-step-train': [], 'running-train': [],
               'per-step-test': [], 'running-test': []}
    R = np.zeros(test_data.shape)
    for cnt, sample in enumerate(pmf_model.trace[burn_in:]):
        sample_R = pmf_model.predict(sample['U'], sample['V'])
        R += sample_R
        running_R = R / (cnt + 1)
        results['per-step-train'].append(rmse(train_data, sample_R))
        results['running-train'].append(rmse(train_data, running_R))
        results['per-step-test'].append(rmse(test_data, sample_R))
        results['running-test'].append(rmse(test_data, running_R))

    results = pd.DataFrame(results)

    if plot:
        results.plot(
            kind='line', grid=False, figsize=(15, 7),
            title='Per-step and Running RMSE From Posterior Predictive')

    # Return the final predictions, and the RMSE calculations
    return running_R, results


PMF.running_rmse = _running_rmse

In [ ]:
predicted, results = pmf.running_rmse(test, train)

In [ ]:
# And our final RMSE?
final_test_rmse = results['running-test'].values[-1]
final_train_rmse = results['running-train'].values[-1]
print('Posterior predictive train RMSE: %.5f' % final_train_rmse)
print('Posterior predictive test RMSE:  %.5f' % final_test_rmse)
print('Train/test difference:           %.5f' % (final_test_rmse - final_train_rmse))
print('Improvement from MAP:            %.5f' % (pmf_map_rmse - final_test_rmse))
print('Improvement from Mean of Means:  %.5f' % (baselines['mom'] - final_test_rmse))